This notebook visualizes closed-loop cost increase vs. switching index.

Order: (a) Differential drive, (b) Drone, (c) Trunk

Highlights tuned switching indices: diff-drive=36, drone=36, trunk=12

In [ ]:
from utils_shared import get_dir
from pathlib import Path
import pickle
import os
data_dir = get_dir("data")
results_file_sweep = data_dir / "differential_drive/differential_drive_sweep.pkl" 
if os.path.exists(results_file_sweep):
    with open(results_file_sweep, 'rb') as f:
        data = pickle.load(f)
    mean_costs_sweep_diff_drive = data['mean_costs_sweep']
    switching_indices_diff_drive = data['switching_indices']
    mean_costs_baseline_diff_drive = data['mean_costs_baseline']
else:
    print("Data not found")

In [ ]:
from utils_shared import get_dir
import pickle
import os
results_file_sweep = data_dir / "drone/drone_sweep.pkl" 
if os.path.exists(results_file_sweep):
    with open(results_file_sweep, 'rb') as f:
        data = pickle.load(f)
    mean_costs_sweep_drone = data['mean_costs_sweep']
    switching_indices_drone = data['switching_indices']
    mean_costs_baseline_drone = data['mean_costs_baseline']
else:
    print("Data not found")

In [ ]:
from utils_shared import get_dir
import pickle
import os
results_file_sweep = data_dir / "trunk/trunk_sweep.pkl" 
if os.path.exists(results_file_sweep):
    with open(results_file_sweep, 'rb') as f:
        data = pickle.load(f)
    mean_costs_sweep_trunk = data['mean_costs_sweep']
    switching_indices_trunk = data['switching_indices']
    mean_costs_baseline_trunk = data['mean_cost_baseline']
else:
    print("Data not found")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerBase
from matplotlib.transforms import blended_transform_factory
from plotting_utils_shared import latexify_plot


# -------- single global font size parameter --------
FS = 22 # <-- change this and everything (axes, ticks, legend, titles) will follow
MS = 3
HSPACE = 0.4  # smaller = tighter vertical spacing
MS_Selected = 11
LW = 1.5
LW_selected = 1.5
plt.rcParams.update({
    "font.size": FS,
    "axes.titlesize": FS,
    "axes.labelsize": FS,
    "xtick.labelsize": FS,
    "ytick.labelsize": FS,
    "legend.fontsize": FS,
})

# ---- y-label pad control (distance between y-axis and the middle subplot's label) ----
YLABEL_PAD = 13  # increase/decrease to move the middle y-label further left/right

# ---- epsilon thresholds (percent) + color for the horizontal lines ----
e_dd, e_drone, e_trunk = 0.05, 0.3, 0.1
EPS_COLOR = 'tab:purple'

# Tuned switching indices
tuned_dd = 21
tuned_dr = 33
tuned_tr = 8

# Helper to highlight the tuned index on an axis (orange PENTAGON, no text)
def highlight_tuned(ax, x, y, tuned_idx, vline_color=None, eps=None):
    x = np.asarray(x); y = np.asarray(y)
    i = int(np.argmin(np.abs(x - tuned_idx)))
    x_star, y_star = x[i], y[i]
    # cropped vertical line: bottom -> epsilon (L-shape corner at (k*, epsilon))
    if eps is None:
        ax.axvline(x_star, linestyle='--', linewidth=LW_selected, color=vline_color, zorder=2)
    else:
        ymin = ax.get_ylim()[0]
        ax.vlines(x_star, ymin=ymin, ymax=eps, linestyle='--',
                  linewidth=LW_selected, color=vline_color, zorder=2)
    # marker above the lines
    ax.plot([x_star], [y_star], marker='p', markersize=MS_Selected,
            mec='k', mew=1.0, mfc='orange', zorder=3)

# --- custom legend handle that shows ONLY a leftward eps-line + downward sel-line behind the pentagon ---
class _SelectedHandle:
    def __init__(self, lw, sel_color, eps_color):
        self.lw = lw
        self.sel_color = sel_color
        self.eps_color = eps_color

class _SelectedHandler(HandlerBase):
    def create_artists(self, legend, orig_handle, x0, y0, w, h, fontsize, trans):
        # horizontal epsilon line (only to the LEFT of the marker)
        l_h = Line2D([x0, x0 + w/2], [y0 + h/2, y0 + h/2],
                     linestyle='--', color=orig_handle.eps_color,
                     linewidth=orig_handle.lw, transform=trans, zorder=1)
        # vertical selected-k line (only BELOW the marker) – extend slightly downward for visibility
        pad = 0.20
        l_v = Line2D([x0 + w/2, x0 + w/2], [y0 - pad*h, y0 + h/2],
                     linestyle='--', color=orig_handle.sel_color,
                     linewidth=orig_handle.lw, transform=trans, zorder=1, clip_on=False)
        # pentagon marker (on top, with extra breathing room)
        m = Line2D([x0 + w/2], [y0 + h/2], marker='p', linestyle='None',
                   markersize=MS_Selected + 3, markerfacecolor='orange', markeredgecolor='black',
                   transform=trans, zorder=2)
        return [l_h, l_v, m]

# 3x1 subplots in the order: diff-drive, drone, trunk
fig, axes = plt.subplots(3, 1, figsize=(8, 12))

latexify_plot(fontsize=FS)

# make plot tighter
fig.set_size_inches(fig.get_size_inches()[0], 6.0)  

# ----- Differential drive -----
ax = axes[0]
x_dd = np.array(switching_indices_diff_drive)[1:]
y_dd = 100 * (np.array(mean_costs_sweep_diff_drive[1:]) - mean_costs_baseline_diff_drive) / mean_costs_baseline_diff_drive
#print(100*(mean_costs_sweep_diff_drive[21]- mean_costs_baseline_diff_drive)/mean_costs_baseline_diff_drive)
ax.plot(x_dd, y_dd, marker='o', linestyle='-', linewidth=LW, markersize=MS)
ax.grid(True, which='major', linestyle='--', alpha=0.6)
ax.set_yscale('log')
sweep_color_dd = ax.lines[0].get_color()
# L-shaped guides
i_dd = int(np.argmin(np.abs(x_dd - tuned_dd)))
x_star_dd = x_dd[i_dd]
x0, x1 = axes[0].get_xlim()
xfrac_dd = (x_star_dd - x0) / (x1 - x0)
axes[0].axhline(e_dd, xmin=0.0, xmax=xfrac_dd,
                linestyle='--', linewidth=LW_selected, color=EPS_COLOR, zorder=1)

# epsilon label at left (BELOW the line, with a slight symmetric gap)
trans_dd = blended_transform_factory(ax.transAxes, ax.transData)
_pad_pts = 3  # adjust if you want a touch more/less space
ax.annotate(rf"$\epsilon={e_dd}$",
            xy=(0.01, e_dd), xycoords=('axes fraction', 'data'),
            xytext=(0, -_pad_pts), textcoords='offset points',
            color=EPS_COLOR, ha='left', va='top')
highlight_tuned(ax, x_dd, y_dd, tuned_dd, vline_color=sweep_color_dd, eps=e_dd)

# Legend (custom handle; slightly larger inner padding)
custom_handle = _SelectedHandle(LW, sel_color=sweep_color_dd, eps_color=EPS_COLOR)
leg = ax.legend(handles=[custom_handle], labels=[r'Selected $\bar{k}$'],
                loc='upper right', bbox_to_anchor=(1, 0.83),
                frameon=True, fancybox=False, framealpha=1.0, edgecolor='black',
                handlelength=1.6, handletextpad=0.28, borderpad=0.28, labelspacing=0.25,
                handler_map={_SelectedHandle: _SelectedHandler()})

# Panel label
axes[0].text(0.98, 0.94, r"\textbf{A. Differential drive robot}",
             transform=axes[0].transAxes, ha='right', va='top',
             bbox=dict(facecolor='white', edgecolor='none', alpha=0.9, boxstyle='round,pad=0.2'))

# ----- Drone -----
ax = axes[1]
x_dr = np.array(switching_indices_drone)[1:]
y_dr = 100 * (np.array(mean_costs_sweep_drone[1:]) - mean_costs_baseline_drone) / mean_costs_baseline_drone
ax.plot(x_dr, y_dr, marker='o', linestyle='-', linewidth=LW, markersize=MS)
ax.set_ylabel(r"Mean closed-loop cost increase [\%]", labelpad=YLABEL_PAD)
ax.grid(True, which='major', linestyle='--', alpha=0.6)
ax.set_yscale('log')
sweep_color_dr = ax.lines[0].get_color()
# L-shaped guides
i_dr = int(np.argmin(np.abs(x_dr - tuned_dr)))
x_star_dr = x_dr[i_dr]
x0, x1 = axes[1].get_xlim()
xfrac_dr = (x_star_dr - x0) / (x1 - x0)
axes[1].axhline(e_drone, xmin=0.0, xmax=xfrac_dr,
                linestyle='--', linewidth=LW_selected, color=EPS_COLOR, zorder=1)
trans_dr = blended_transform_factory(ax.transAxes, ax.transData)
ax.text(0.01, e_drone, rf"$\epsilon={e_drone}$", color=EPS_COLOR,
        transform=trans_dr, ha='left', va='bottom')
highlight_tuned(ax, x_dr, y_dr, tuned_dr, vline_color=sweep_color_dr, eps=e_drone)

axes[1].text(0.98, 0.94, r"\textbf{B. Drone with pendulum}",
             transform=axes[1].transAxes, ha='right', va='top',
             bbox=dict(facecolor='white', edgecolor='none', alpha=0.9, boxstyle='round,pad=0.2'))

# ----- Trunk -----
ax = axes[2]
x_tr = np.array(switching_indices_trunk)[1:]
y_tr = 100 * (np.array(mean_costs_sweep_trunk[1:]) - mean_costs_baseline_trunk) / mean_costs_baseline_trunk
ax.plot(x_tr, y_tr, marker='o', linestyle='-', linewidth=LW, markersize=MS)
ax.set_xlabel(r"Switching stage $\bar{k}$")
ax.grid(True, which='major', linestyle='--', alpha=0.6)
ax.set_yscale('log')
sweep_color_tr = ax.lines[0].get_color() 
# L-shaped guides
i_tr = int(np.argmin(np.abs(x_tr - tuned_tr)))
x_star_tr = x_tr[i_tr]
x0, x1 = axes[2].get_xlim()
xfrac_tr = (x_star_tr - x0) / (x1 - x0)
axes[2].axhline(e_trunk, xmin=0.0, xmax=xfrac_tr,
                linestyle='--', linewidth=LW_selected, color=EPS_COLOR, zorder=1)
trans_tr = blended_transform_factory(ax.transAxes, ax.transData)
# epsilon label at left (ABOVE the line — reverted)
ax.text(0.01, e_trunk, rf"$\epsilon={e_trunk}$", color=EPS_COLOR,
        transform=trans_tr, ha='left', va='bottom')
highlight_tuned(ax, x_tr, y_tr, tuned_tr, vline_color=sweep_color_tr, eps=e_trunk)

axes[2].text(0.98, 0.94, r"\textbf{C. Trunk-like system}",
             transform=axes[2].transAxes, ha='right', va='top',
             bbox=dict(facecolor='white', edgecolor='none', alpha=0.9, boxstyle='round,pad=0.2'))

plt.tight_layout()
fig.subplots_adjust(hspace=HSPACE)

# save the generated plot
out_dir = get_dir("plots/other")
out_dir.mkdir(parents=True, exist_ok=True)
fig.savefig(out_dir / "Closed_loop_costs_vs_switching_index.pdf", bbox_inches="tight")

plt.show()



Visualize step size schedules

In [ ]:
from utils_shared import compute_exponential_step_sizes
import numpy as np
import math

N_dd = 80
T_dd = 10
dt_initial_dd = 0.01

step_sizes_dd = compute_exponential_step_sizes(
    dt_initial=dt_initial_dd,
    T_total=T_dd,
    N_steps=N_dd,
    plot=True
)

print(f"DD: The (approx.) corresponding switching index with uniform schedule is: {math.ceil(sum(step_sizes_dd[:tuned_dd])/dt_initial_dd)}")

N_drone = 75
dt_initial_drone = 0.01 
T_drone = dt_initial_drone*150

step_sizes_drone = compute_exponential_step_sizes(
    dt_initial=dt_initial_drone,
    T_total=T_drone,
    N_steps=N_drone,
    plot=True
)

print(f"Drone: The (approx.) corresponding switching index with uniform schedule is: {math.ceil(sum(step_sizes_drone[:tuned_dr])/dt_initial_drone)}")

N_trunk = 25
dt_initial_trunk = 0.005
T_trunk = dt_initial_trunk*40

step_sizes_trunk = compute_exponential_step_sizes(
    dt_initial=dt_initial_trunk,
    T_total=T_trunk,
    N_steps=N_trunk,
    plot=True
)

print(f"Trunk: The (approx.) corresponding switching index with uniform schedule is: {math.ceil(sum(step_sizes_trunk[:tuned_tr])/dt_initial_trunk)}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from plotting_utils_shared import latexify_plot

# ---------- style (same as before) ----------
FS = 22
MS = 3
LW = 1.5
HSPACE = 0.30  # a bit more vertical breathing room than the main-figure
plt.rcParams.update({
    "font.size": FS,
    "axes.titlesize": FS,
    "axes.labelsize": FS,
    "xtick.labelsize": FS,
    "ytick.labelsize": FS,
    "legend.fontsize": FS,
})

latexify_plot(fontsize=FS)

# ---------- x-axes (stage index) ----------
k_dd     = np.arange(1, len(step_sizes_dd) + 1)
k_drone  = np.arange(1, len(step_sizes_drone) + 1)
k_trunk  = np.arange(1, len(step_sizes_trunk) + 1)

# ---------- figure ----------
fig, axes = plt.subplots(3, 1, figsize=(8, 12))
fig.set_size_inches(fig.get_size_inches()[0], 7.0)  # less squeezed than the main plot

# ----- (A) Differential drive -----
ax = axes[0]
ax.plot(k_dd, step_sizes_dd, marker='o', linestyle='-', linewidth=LW, markersize=MS)
ax.grid(True, which='major', linestyle='--', alpha=0.6)
# panel label in UPPER LEFT
ax.text(0.02, 0.95, r"\textbf{A. Differential drive robot}",
        transform=ax.transAxes, ha='left', va='top',
        bbox=dict(facecolor='white', edgecolor='none', alpha=0.9, boxstyle='round,pad=0.2'))
#ax.set_yscale('log')

# ----- (B) Drone -----
ax = axes[1]
ax.plot(k_drone, step_sizes_drone, marker='o', linestyle='-', linewidth=LW, markersize=MS)
ax.grid(True, which='major', linestyle='--', alpha=0.6)
ax.set_ylabel(r"Step size $\Delta t$ [s]")  # single shared y-label (linear scale)
ax.text(0.02, 0.95, r"\textbf{B. Drone with pendulum}",
        transform=ax.transAxes, ha='left', va='top',
        bbox=dict(facecolor='white', edgecolor='none', alpha=0.9, boxstyle='round,pad=0.2'))

# ----- (C) Trunk -----
ax = axes[2]
ax.plot(k_trunk, step_sizes_trunk, marker='o', linestyle='-', linewidth=LW, markersize=MS)
ax.grid(True, which='major', linestyle='--', alpha=0.6)
ax.set_xlabel(r"Stage $k$")
ax.text(0.02, 0.95, r"\textbf{C. Trunk-like system}",
        transform=ax.transAxes, ha='left', va='top',
        bbox=dict(facecolor='white', edgecolor='none', alpha=0.9, boxstyle='round,pad=0.2'))

plt.tight_layout()
fig.subplots_adjust(hspace=HSPACE)

# save
out_dir = get_dir("plots/other")
out_dir.mkdir(parents=True, exist_ok=True)
fig.savefig(out_dir / "step_sizes_vs_stage.pdf", bbox_inches="tight")

plt.show()


